In [ ]:
CREATE OR REPLACE TABLE project2.raw_data.grocery_price(
record_id integer primary key,
market_id integer,
market varchar(32),
item_no integer,
item_name varchar(32),
sale_unit varchar(32),
price_krw float,
year_month varchar(32),
note varchar,
market_type_code number,
market_type_name varchar(32),
district_code number,
district_name varchar(32),
inspection_date date
);

COPY INTO project2.raw_data.grocery_price
from 's3://yeonkyu-test-bucket/grocery_price.csv'
credentials=(AWS_KEY_ID='***' AWS_SECRET_KEY='***')
FILE_FORMAT = (type = 'CSV' skip_header=1 FIELD_OPTIONALLY_ENCLOSED_BY='"');

SELECT *
FROM project2.analytics.grocery_price_analytics;

SELECT DISTRICT_NAME, round(avg(PRICE_KRW)) AS GROCERY_AVG_PRICE
FROM project2.raw_data.grocery_price
WHERE PRICE_KRW <> 0
GROUP BY DISTRICT_NAME
ORDER BY PRICE;

CREATE OR REPLACE TABLE project2.analytics.grocery_price_analytics AS
SELECT a."자치구명", a."평단가", b.GROCERY_AVG_PRICE
FROM project2.analytics.property_unit_price a
JOIN (
    SELECT DISTRICT_NAME, ROUND(AVG(PRICE_KRW)) AS GROCERY_AVG_PRICE
    FROM project2.raw_data.grocery_price
    WHERE PRICE_KRW <> 0
    GROUP BY DISTRICT_NAME
) b
ON a."자치구명" = b.DISTRICT_NAME
WHERE a."접수연도" = 2023
ORDER BY b.GROCERY_AVG_PRICE;

